# Lab 2

In [63]:
import os
from PIL import Image
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from typing import Tuple


## Локальная пороговая обработка



### Метод Niblack 



Метод используется на практике для быстрой фильтрации контрастных изображений, на которых отсутствуют сильно зашумленные области с плавными
переходами яркости. 

- μ(x, y) – среднее и s(x, y) — среднеквадратичное отклонение выборки для некоторой окрестности точки.
- Значение k определяет, какую часть границы объекта взять в качестве самого объекта. 

In [56]:
def niblack(img, k,n=5):

    dx, dy = img.shape
    img_copy = img.copy()

    r = (n - 1) // 2

    for i in range(r, dx - r):
        
        for j in range(r, dy - r):

            block = img[i - r:i + r + 1, j - r:j + r + 1]

            mean = np.sum(block) / (n * n)
            rBstd = np.sqrt(np.sum((block - mean) ** 2) / (n * n))
        
            threshold = mean + k * rBstd
            
            if img[i, j] < threshold:
                img_copy[i, j] = 0
            else:
                img_copy[i, j] = 255

    return img_copy
    

### Sauvola Method

- k is a control factor in the range of [0.2, 0.5] 
- R is a predetermined image graylevel value.
- s is a standard deviation of the neighbourhood

$$
T(x, y) = \mu(x, y) \cdot \left[ 1 - k \cdot \left(1 - \frac{s(x, y)}{R} \right) \right]
$$

In [57]:
def sauvola(img, n=5, R=128, k=0.5):
    
    dx, dy = img.shape
    imgN = np.copy(img)

    r = (n - 1) // 2

    for i in range(r, dx - r):
        for j in range(r, dy - r):

            block = img[i - r:i + r + 1, j - r:j + r + 1]

            rBmn = np.mean(block)
            rBstd = np.std(block)

            rBTH = rBmn * (1 - k * (1 - (rBstd / R)))
            if (img[i,j] < rBTH):
                imgN[i,j] = 0
            else:
                imgN[i,j] = 255

    return imgN


### Сhristian threshold

In [66]:
import numpy as np

def christian(img, n=5, k=0.5):

    dx, dy = img.shape
    imgN = np.copy(img)

    M = np.min(img)
    R = 0  
    
    for x in range(n // 2, dx - n // 2):
        for y in range(n // 2, dy - n // 2):

            block = img[x - (n // 2): x + (n // 2) + 1, y - (n // 2): y + (n // 2) + 1]

            block_std = np.std(block)
   
            if block_std > R:
                R = block_std

    r = (n - 1) // 2

    for i in range(r, dx - r):
        for j in range(r, dy - r):

            block = img[i - r:i + r + 1, j - r:j + r + 1]

            rBmn = np.mean(block)
            rBstd = np.std(block)

            rBTH = (1 - k) * rBmn + k * M + k * (rBstd / R) * (rBmn - M)
 
            if img[i, j] < rBTH:
                imgN[i, j] = 0
            else:
                imgN[i, j] = 255

    return imgN


## Prepare to lab

In [59]:
filepath = "before/text_foto_lab1.png"

In [60]:
def resize_rith_pad(image: np.array, ner_shape: Tuple[int, int], padding_color: Tuple[int] = (255, 255, 255)) -> np.array:
      original_shape = (image.shape[1], image.shape[0])
      ratio = float(max(ner_shape))/max(original_shape)
      ner_size = tuple([int(x*ratio) for x in original_shape])
  
      if ner_size[0] > ner_shape[0] or ner_size[1] > ner_shape[1]:
          ratio = float(min(ner_shape)) / min(original_shape)
          ner_size = tuple([int(x * ratio) for x in original_shape])
  
      image = cv2.resize(image, ner_size)
      delta_r = ner_shape[0] - ner_size[0]
      delta_h = ner_shape[1] - ner_size[1]
      top, bottom = delta_h//2, delta_h-(delta_h//2)
      left, right = delta_r//2, delta_r-(delta_r//2)
  
      image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT,None,value=padding_color)
      return image # rill be use in the future)) 

In [61]:
# image = cv2.imread(filepath)
# image = resize_rith_pad(image, (256, 256))
    
# cv2.imshor("Padded image", image)
# cv2.raitKey()

## Result

In [67]:
img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

niblack_img = niblack(img,-0.2)
cv2.imwrite('niblack_img.png', niblack_img)

sauvola_img= sauvola(img)
cv2.imwrite('sauvola_img.png', sauvola_img)

christian_img = christian(img, )
cv2.imwrite('christian_img.png', christian_img)

True